In [ ]:
from astropy.table import Table, join
import numpy as np
import matplotlib.pyplot as plt

### Load catalogs

In [ ]:
# Load the raw catalog
raw = Table.read("../data/dp1_with_photoz_v2.fits")
raw.remove_column("objectId")  # Remove duplicated ID column

# Select ECDFS region only
# raw = raw[
#    (raw["coord_ra"] >= 52) & (raw["coord_ra"] <= 55) & (raw["coord_dec"] <= -27)
#    & (raw["coord_dec"] >= -29.0)
# ]

# Save photo-z estimate that is weighted mean of FlexZBoost and LePhare
fzb = raw["fzboost_z_mode"]
fzb_sig = (raw["fzboost_z_err68_high"] - raw["fzboost_z_err68_low"]) / 2
lph = raw["lephare_z_mode"]
lph_sig = (raw["lephare_z_err68_high"] - raw["lephare_z_err68_low"]) / 2
raw["z_phot"] = (fzb / fzb_sig**2 + lph / lph_sig**2) / (
    1 / fzb_sig**2 + 1 / lph_sig**2
)

# Load the redshift reference catalog
zref = Table.read("../data/dp1_ecdfs_crossmatched.parquet")
zref = zref[np.isfinite(zref["redshift"]) & (zref["confidence"] >= 0.95)]
zref = zref[["objectId", "redshift", "confidence", "type", "source"]]

# Join these catalogs
raw = join(raw, zref, join_type="left", keys_left="objectID", keys_right="objectId")

### Cuts on photo-z

In [ ]:
print(f"Size of raw catalog: {len(raw)} galaxies")

# Define foreground and background redshift ranges
foreground = dict(zmin=0.2, zmax=0.5)
background = dict(zmin=0.6, zmax=1.5)

# SNR > 5 in grz (SNR > 10 already applied in i)
cut = (
    (raw["g_cModelFlux"] / raw["g_cModelFluxErr"] > 5)
    & (raw["r_cModelFlux"] / raw["r_cModelFluxErr"] > 5)
    & (raw["z_cModelFlux"] / raw["z_cModelFluxErr"] > 5)
)

# High-quality photo-z in the foreground & reliable tomography in background
fzb = raw["fzboost_z_mode"]
lph = raw["lephare_z_mode"]
cut &= (
    (np.abs(fzb - lph) < 0.2 * (1 + raw["z_phot"]))
    & ((raw["fzboost_z_err68_high"] - raw["fzboost_z_err68_low"]) / 2 < 0.25)
    & ((raw["lephare_z_err68_high"] - raw["lephare_z_err68_low"]) / 2 < 0.25)
)  # | (
#    (fzb > background["zmin"]) & (lph > background["zmin"])
# )

# Apply cut
cat = raw[cut]

print(f"After photo-z cuts: {len(cat)} galaxies")

fg_cat = cat[
    (cat["z_phot"] >= foreground["zmin"]) & (cat["z_phot"] <= foreground["zmax"])
]
bg_cat = cat[
    (cat["z_phot"] >= background["zmin"]) & (cat["z_phot"] <= background["zmax"])
]

# Select red-sequence galaxies in background
# r = -2.5 * np.log10(bg_cat["r_gaap1p0Flux"])
# i = -2.5 * np.log10(bg_cat["i_gaap1p0Flux"])
# z_cmodel = -2.5 * np.log10(bg_cat["z_cModelFlux"]) + 31.4
# mask = ((r - i) > -0.1 * z_cmodel + 3.1) & ((r - i) < -0.1 * z_cmodel + 3.5)
# bg_cat = bg_cat[mask]

print(f"\n{len(fg_cat)} foreground galaxies")
print(f"{len(bg_cat)} background galaxies")

fg_cat.write("../data/foreground_catalog.fits", overwrite=True)
bg_cat.write("../data/background_catalog.fits", overwrite=True)

### Plots

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(7, 2), dpi=150)

for ax in (ax1, ax2, ax3):
    ax.set(xlabel="Spec-z", xlim=(0, 2), ylim=(0, 2), aspect="equal")
    ax.plot([0, 2], [0, 2], c="k", lw=1, ls="--")

ax1.scatter(raw["redshift"], raw["fzboost_z_mode"], s=0.5)
ax1.set(ylabel="FlexZBoost photo-z")

ax2.scatter(raw["redshift"], raw["lephare_z_mode"], s=0.5)
ax2.set(ylabel="LePhare photo-z")

ax3.scatter(cat["redshift"], cat["z_phot"], s=0.5)
ax3.set(ylabel="Combined photo-z")

fig.subplots_adjust(wspace=0.4)

fig.savefig("../figures/photoz.png", dpi=500, bbox_inches="tight")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(7, 2), dpi=150)

settings = dict(extent=(0, 2, 0, 2), gridsize=50, norm="log", edgecolors="none")
ax1.hexbin(fg_cat["redshift"], fg_cat["z_phot"], cmap="Blues", **settings)
ax1.hexbin(bg_cat["redshift"], bg_cat["z_phot"], cmap="Reds", **settings)
ax1.set(
    xlim=(0, 1.5),
    ylim=(0, 1.5),
    xlabel="Spec-z",
    ylabel="Photo-z",
    aspect="equal",
)
ax1.plot([0, 2], [0, 2], c="k", lw=1, ls="--", zorder=0)

ax2.hexbin(fg_cat["fzboost_z_mode"], fg_cat["lephare_z_mode"], cmap="Blues", **settings)
ax2.hexbin(bg_cat["fzboost_z_mode"], bg_cat["lephare_z_mode"], cmap="Reds", **settings)
ax2.set(
    xlim=(0, 1.5),
    ylim=(0, 1.5),
    xlabel="FlexZBoost photo-z",
    ylabel="LePhare photo-z",
    aspect="equal",
)
ax2.plot([0, 2], [0, 2], c="k", lw=1, ls="--", zorder=0)

# Plot redshift histograms

# Settings for all histograms
settings = dict(range=(0, 1.5), bins=50, density=True, histtype="step")

# Foreground sample
ax3.hist(
    fg_cat["z_phot"],
    **settings,
    color="C0",
    ls="-",
)
ax3.hist(
    fg_cat["redshift"][~fg_cat["redshift"].mask],
    **settings,
    color="C0",
    ls="--",
)

# Background sample
ax3.hist(
    bg_cat["z_phot"],
    **settings,
    color="C3",
    ls="-",
)
ax3.hist(
    bg_cat["redshift"][~bg_cat["redshift"].mask],
    **settings,
    color="C3",
    ls="--",
)

ax3.set(
    xlim=(0, 1.5),
    ylim=(0, 6.5),
    xlabel="Redshift",
    ylabel="Frequency",
    aspect=1.5 / 6.5,
)

ax3.hist([], histtype="step", color="k", ls="-", label="Photo-z")
ax3.hist([], histtype="step", color="k", ls="--", label="Spec-z")
ax3.legend(handlelength=1, frameon=False, fontsize=8)

fig.subplots_adjust(wspace=0.4)

fig.savefig("../figures/photoz_cuts.png", dpi=500, bbox_inches="tight")